In [150]:
import numpy as np
import os
import boto3
import tensorflow as tf

In [151]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

In [152]:
bucket = 'demo-saeed'
prefix = 'fraudcredit-keras1_8m_11'
dataset_train_name = 'creditcard1_8m_train.csv'
dataset_test_name='creditcard1_8m_test.csv'

In [153]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'data/train/{}'.format(dataset_train_name))).upload_file('data/{}'.format(dataset_train_name))


In [5]:
#boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'data/test/{}'.format(dataset_test_name))).upload_file('data/{}'.format(dataset_test_name))


In [6]:
%%bash

for i in {0..8}; do
    aws s3 cp s3://demo-saeed/fraudcredit-keras1_8m_9/data/test/creditcard1_8m_test.csv s3://demo-saeed/fraudcredit-keras1_8m_9/data/test/creditcard1_8m_test_${i}.csv 
done

Process is terminated.


In [8]:
# train_input = sagemaker_session.upload_data('data', key_prefix="{}/{}/{}".format(bucket_name, prefix, 'data') )
# train_input

In [154]:
data_location = 's3://{}/{}/{}'.format(bucket, prefix, 'data/train',dataset_train_name)
s3_input_train = sagemaker.s3_input(s3_data = data_location, content_type='csv')
s3_input_train.config

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
   'S3Uri': 's3://demo-saeed/fraudcredit-keras1_8m_11/data/train',
   'S3DataDistributionType': 'FullyReplicated'}},
 'ContentType': 'csv'}

In [155]:
train_input = 's3://{}/{}/data/train/{}'.format(bucket,prefix,dataset_train_name)
train_input

's3://demo-saeed/fraudcredit-keras1_8m_11/data/train/creditcard1_8m_train.csv'

In [156]:
# import pandas as pd
# df = pd.read_csv(train_input, sep=',',  nrows=10)
# df.head()

In [157]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='main.py', 
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='ml.p3.2xlarge', #ml.p3.2xlarge
                          framework_version='1.13', 
                          py_version='py3',
                          script_mode=True,
                          hyperparameters={'epochs': 1}
                         )  


In [158]:
tf_estimator.fit({'train': s3_input_train})

2020-03-09 20:48:12 Starting - Starting the training job...
2020-03-09 20:48:14 Starting - Launching requested ML instances......
2020-03-09 20:49:19 Starting - Preparing the instances for training......
2020-03-09 20:50:18 Downloading - Downloading input data.........
2020-03-09 20:51:58 Training - Downloading the training image..2020-03-09 20:52:21,970 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-03-09 20:52:22,430 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "model_dir": "s3://sagemaker-us-east-1-079329190341/tensorflow-training-2020-03-09-20-48-12-031/model",
        "epochs": 1
    },
    "input_config_dir": "/opt/ml/i

In [172]:
tf_estimator.model_dir

's3://sagemaker-us-east-1-079329190341/tensorflow-training-2020-03-09-20-48-12-031/model'

# Prediciton using endpoint high-level sdk

In [160]:
# import time

# tf_endpoint_name = 'keras-tf-fmnist-'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
# tf_predictor = tf_estimator.deploy(initial_instance_count=1,
#                          instance_type='local',        # $0.134/hour in eu-west-1
#                          endpoint_name=tf_endpoint_name)     # = 80% discount!

In [161]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split
# RANDOM_SEED = 42
# LABELS = ["Normal", "Fraud"]
# data = df.drop(['Time'], axis=1)

# data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))
# X_train, X_test = train_test_split(data, test_size=0.2, random_state=RANDOM_SEED)
# X_train = X_train[X_train.Class == 0]
# X_train = X_train.drop(['Class'], axis=1)

# y_test = X_test['Class']
# X_test = X_test.drop(['Class'], axis=1)

# batch_file = 'batch_data.csv'
# X_test.to_csv(batch_file,index=False,header=False)
# #sess.upload_data(batch_file, key_prefix='{}/batch'.format(prefix))
# boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'batch/batch_data.csv')).upload_file('batch_data.csv')

# X_train = X_train.values
# X_test = X_test.values
# print(X_train.shape)

In [162]:
# predictions = tf_predictor.predict(X_test)['predictions']
# mse = np.mean(np.power(X_test - predictions, 2), axis=1)
# error_df = pd.DataFrame({'reconstruction_error': mse,
#                         'true_class': y_test})
# error_df.head(20)

# Prediciton using Tensorflow Serving

In [163]:
# !aws s3 cp s3://sagemaker-us-east-1-079329190341/sagemaker-tensorflow-scriptmode-2020-03-06-16-23-30-671/model.tar.gz model.tar.gz
# !tar -xf model.tar.gz

In [164]:
# sess = tf.compat.v1.Session()
# loaded = tf.saved_model.load(export_dir='SavedModels/1/', tags={'serve'},sess=sess )
# loaded.signature_def

In [222]:
!saved_model_cli show --dir SavedModels/autoencodermodel/1/ --tag_set serve --all



MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 2108909)
        name: input_1_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_4_1/Relu:0'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 2108909)
        name: dense_4_1/Relu:0
  Method name is: tensorflow/serving/predict


In [210]:
%%bash
mkdir -p tmp/tfserving
cd tmp/tfserving
git clone --depth=1 https://github.com/tensorflow/serving
cd tmp/tfserving
docker pull tensorflow/serving:latest

latest: Pulling from tensorflow/serving
Digest: sha256:ea44bf657f8cff7b07df12361749ea94628185352836bb08065345f5c8284bae
Status: Image is up to date for tensorflow/serving:latest


fatal: destination path 'serving' already exists and is not an empty directory.
bash: line 4: cd: tmp/tfserving: No such file or directory


In [217]:
!docker kill $(docker ps -q)
!docker run --rm -p 8501:8501 \
    --mount type=bind,source=$(pwd),target=$(pwd) \
    -e MODEL_BASE_PATH=$(pwd)/SavedModels \
    -e MODEL_NAME=autoencodermodel -t tensorflow/serving:latest

2020-03-10 00:43:33.109627: I tensorflow_serving/model_servers/server.cc:86] Building single TensorFlow model file config:  model_name: autoencodermodel model_base_path: /home/ec2-user/SageMaker/Saeed-demo/Credit-Card-Fraud-Detection-using-Autoencoders-in-Keras/SavedModels/autoencodermodel
2020-03-10 00:43:33.109769: I tensorflow_serving/model_servers/server_core.cc:462] Adding/updating models.
2020-03-10 00:43:33.109784: I tensorflow_serving/model_servers/server_core.cc:573]  (Re-)adding model: autoencodermodel
2020-03-10 00:43:33.210056: I tensorflow_serving/core/basic_manager.cc:739] Successfully reserved resources to load servable {name: autoencodermodel version: 1}
2020-03-10 00:43:33.210081: I tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: autoencodermodel version: 1}
2020-03-10 00:43:33.210097: I tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: autoencodermodel version: 1}
2020-03-10 00:43:33.210116: I extern

In [218]:
!docker ps

CONTAINER ID        IMAGE                       COMMAND                  CREATED             STATUS              PORTS                              NAMES
22c591abbd1a        tensorflow/serving:latest   "/usr/bin/tf_serving…"   13 seconds ago      Up 12 seconds       8500/tcp, 0.0.0.0:8501->8501/tcp   quirky_haslett


In [24]:
# %%bash
# cd /tmp/tfserving/
# curl http://localhost:8501/v1/models/autoencodermodel

In [25]:
# !curl -d '{"instances": [[1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0], \
# [1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0]]}' -X POST http://localhost:8501/v1/models/autoencodermodel:predict

# Prediction using Batch Transform

In [169]:
tf_estimator.model_dir

's3://sagemaker-us-east-1-079329190341/tensorflow-training-2020-03-09-20-48-12-031/model'

In [173]:
%%bash

aws s3 cp s3://sagemaker-us-east-1-079329190341/tensorflow-training-2020-03-09-20-48-12-031/output/model.tar.gz model.tar.gz


download: s3://sagemaker-us-east-1-079329190341/tensorflow-training-2020-03-09-20-48-12-031/output/model.tar.gz to ./model.tar.gz


In [219]:
%%bash
tar -xvf model.tar.gz
rm -rd SavedModels/autoencodermodel/1
cp -r 1 SavedModels/autoencodermodel/1/

1/
1/saved_model.pb
1/variables/
1/variables/variables.index
1/variables/variables.data-00000-of-00001


In [227]:
%%bash
pwd
cd SavedModels/
tar -czvf model.tar.gz code --directory=autoencodermodel 1

/home/ec2-user/SageMaker/Saeed-demo/Credit-Card-Fraud-Detection-using-Autoencoders-in-Keras
code/
code/requirements.txt
code/.ipynb_checkpoints/
code/.ipynb_checkpoints/requirements-checkpoint.txt
code/.ipynb_checkpoints/inference-checkpoint.py
code/inference.py
1/
1/saved_model.pb
1/variables/
1/variables/variables.index
1/variables/variables.data-00000-of-00001


In [221]:
!ls -al SavedModels/autoencodermodel/1

total 272
drwxr-xr-x 3 ec2-user ec2-user   4096 Mar 10 00:54 .
drwxrwxr-x 4 ec2-user ec2-user   4096 Mar 10 00:54 ..
-rw-r--r-- 1 ec2-user ec2-user 263494 Mar 10 00:54 saved_model.pb
drwxr-xr-x 2 ec2-user ec2-user   4096 Mar 10 00:54 variables


In [228]:
from sagemaker.tensorflow.serving import Model
model_data = sagemaker_session.upload_data('SavedModels/model.tar.gz',
                                           bucket,
                                           os.path.join(prefix, 'model'))
                                           
tensorflow_serving_model = Model(model_data=model_data,
                                 role=role,
                                 framework_version='1.13',
                                 sagemaker_session=sagemaker_session)

In [204]:
model_data

's3://demo-saeed/fraudcredit-keras1_8m_11/model/model.tar.gz'

In [205]:
output_path='s3://{}/{}/batch_output/'.format(bucket, prefix)
output_path

's3://demo-saeed/fraudcredit-keras1_8m_11/batch_output/'

In [229]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'data/test4/creditcard1_8m_test.tfrecords')).upload_file('data/creditcard1_8m_test.tfrecords')

In [230]:
!aws s3 ls s3://demo-saeed/fraudcredit-keras1_8m_11/data/test4/

2020-03-10 01:41:13  843568000 creditcard1_8m_test.tfrecords


In [233]:
%%bash

for i in {0..8}; do
    aws s3 cp s3://demo-saeed/fraudcredit-keras1_8m_11/data/test4/creditcard1_8m_test.tfrecords s3://demo-saeed/fraudcredit-keras1_8m_11/data/test4/creditcard1_8m_test_${i}.tfrecords 
done

copy: s3://demo-saeed/fraudcredit-keras1_8m_11/data/test4/creditcard1_8m_test.tfrecords to s3://demo-saeed/fraudcredit-keras1_8m_11/data/test4/creditcard1_8m_test_0.tfrecords
copy: s3://demo-saeed/fraudcredit-keras1_8m_11/data/test4/creditcard1_8m_test.tfrecords to s3://demo-saeed/fraudcredit-keras1_8m_11/data/test4/creditcard1_8m_test_1.tfrecords
copy: s3://demo-saeed/fraudcredit-keras1_8m_11/data/test4/creditcard1_8m_test.tfrecords to s3://demo-saeed/fraudcredit-keras1_8m_11/data/test4/creditcard1_8m_test_2.tfrecords
copy: s3://demo-saeed/fraudcredit-keras1_8m_11/data/test4/creditcard1_8m_test.tfrecords to s3://demo-saeed/fraudcredit-keras1_8m_11/data/test4/creditcard1_8m_test_3.tfrecords
copy: s3://demo-saeed/fraudcredit-keras1_8m_11/data/test4/creditcard1_8m_test.tfrecords to s3://demo-saeed/fraudcredit-keras1_8m_11/data/test4/creditcard1_8m_test_4.tfrecords
copy: s3://demo-saeed/fraudcredit-keras1_8m_11/data/test4/creditcard1_8m_test.tfrecords to s3://demo-saeed/fraudcredit-keras1

In [ ]:
env = {'SAGEMAKER_TFS_ENABLE_BATCHING': 'true', 'SAGEMAKER_TFS_BATCH_TIMEOUT_MICROS': '50000','SAGEMAKER_TFS_MAX_BATCH_SIZE': '16'}

#sm_transformer = tf_estimator.transformer
sm_transformer = tensorflow_serving_model.transformer(instance_count=2,
                                          instance_type='ml.p3.2xlarge', #ml.p3.2xlarge
                                          max_concurrent_transforms=5,
                                          max_payload=20,
                                          strategy='SingleRecord', #MultiRecord
                                          #env=env,
                                          output_path='s3://{}/{}/batch_output/'.format(bucket, prefix))
# start a transform job
input_location = 's3://{}/{}/data/test4'.format(bucket, prefix) # use input data without ID column
sm_transformer.transform(input_location, 
                         data_type='S3Prefix',
                         content_type= "application/x-tfexample", #'application/x-recordio-protobuf', 'text/csv'
                         split_type='TFRecord' )#RecordIO
sm_transformer.wait()

Using already existing model: tensorflow-inference-2020-03-10-01-41-29-595


........................INFO:__main__:starting services
INFO:__main__:using default model name: model
INFO:__main__:tensorflow serving model config: 
model_config_list: {
  config: {
    name: "model",
    base_path: "/opt/ml/model",
    model_platform: "tensorflow"
  },
}


INFO:__main__:nginx config: 
load_module modules/ngx_http_js_module.so;

worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr error;

worker_rlimit_nofile 4096;

events {
  worker_connections 2048;
}

http {
  include /etc/nginx/mime.types;
  default_type application/json;
  access_log /dev/stdout combined;
  js_include tensorflow-serving.js;

  upstream tfs_upstream {
    server localhost:10001;
  }

  upstream gunicorn_upstream {
    server unix:/tmp/gunicorn.sock fail_timeout=1;
  }

  server {
    listen 8080 deferred;
    client_max_body_size 0;
    client_body_buffer_size 100m;
    subrequest_output_buffer_size 100m;

    set $tfs_version 1.13;
    set $default_tfs_model model;

    l

In [ ]:
output_path

In [ ]:
# import json
# import io
# from urllib.parse import urlparse

# def get_csv_output_from_s3(s3uri, file_name):
#     parsed_url = urlparse(s3uri)

#     bucket_name = parsed_url.netloc

#     prefix = parsed_url.path[1:]

#     s3 = boto3.resource('s3')
#     obj = s3.Object(bucket_name, '{}/{}'.format(prefix, file_name))

#     return obj.get()["Body"].read().decode('utf-8')

In [ ]:
# output = get_csv_output_from_s3(output_path[:-1], dataset_test_name+'.out')
# j =json.loads(output)
# n =j['predictions']
# df = pd.DataFrame.from_records(n)
# df.head()